In [2]:
import cv2
import numpy as np
import os
from scipy.ndimage import label

In [8]:
def process_image(image_path, output_folder):
    # Load the image using OpenCV
    original_image = cv2.imread(image_path)

    # Ensure the image was properly loaded
    if original_image is not None:
        # Make a copy of the original image for processing
        modified_image = np.copy(original_image)

        # Your provided code for image processing
        for i in range(modified_image.shape[0]):
            for j in range(modified_image.shape[1]):
                if modified_image[i, j, 1] <= 70 or modified_image[i, j, 1] >= 200:  # Green component
                    modified_image[i, j] = [0, 0, 0]

                if modified_image[i, j, 2] > 165:  # r component
                    modified_image[i, j] = [0, 0, 0]

                if modified_image[i, j, 0] > 90:  # b component
                    modified_image[i, j] = [0, 0, 0]

        output_subfolder = os.path.join(output_folder, 'originals/')
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)

        # Save the modified image and original image in the output folder
        filename = os.path.splitext(os.path.basename(image_path))[0]
        cv2.imwrite(os.path.join(output_subfolder, f'{filename}_original.png'), original_image)
        cv2.imwrite(os.path.join(output_folder, f'{filename}.jpg'), modified_image)
    else:
        print(f"Image {image_path} not found or couldn't be loaded.")

# Path to the folder containing multiple images
input_folder = 'inputs/'

# Output folder for processed images
output_folder = '1-final/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Process only image files
        image_path = os.path.join(input_folder, filename)
        process_image(image_path, output_folder)

In [9]:
def process_image_hsv(image_path, output_folder):
    # Load the image using OpenCV
    new_image = cv2.imread(image_path)

    # Ensure the image was properly loaded
    if new_image is not None:
        # Convert the image to the HSV color space
        hsv = cv2.cvtColor(new_image, cv2.COLOR_BGR2HSV)

        # Define the saturation range
        lower_saturation = np.array([28, 50, 15])
        upper_saturation = np.array([255, 255, 255])

        # Apply thresholding to the HSV image to get colors with saturation greater than 25
        mask = cv2.inRange(hsv, lower_saturation, upper_saturation)

        # Apply bitwise AND operation between the mask and the original image
        result = cv2.bitwise_and(new_image, new_image, mask=mask)


        # Get the filename without extension
        filename = os.path.splitext(os.path.basename(image_path))[0]

        # Save the filtered image
        filtered_image_path = os.path.join(output_folder, f'{filename}.jpg')
        cv2.imwrite(filtered_image_path, result)
        
    else:
        print(f"Image {image_path} not found or couldn't be loaded.")

# Path to the folder containing multiple images
input_folder = '1-final/'

# Output folder for processed images
output_folder = '2-final/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Process only image files
        image_path = os.path.join(input_folder, filename)
        process_image_hsv(image_path, output_folder)

In [24]:
def process_image_binary(image_path, output_folder):
    # Load the image using OpenCV
    new_image = cv2.imread(image_path)

    # Ensure the image was properly loaded
    if new_image is not None:
        # Convert the image to grayscale
        gray = cv2.cvtColor(new_image, cv2.COLOR_BGR2GRAY)

        # Define the threshold value
        threshold_value = 60

        # Apply thresholding to the grayscale image
        _, binary_image = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY)

        # Get the filename without extension
        filename = os.path.splitext(os.path.basename(image_path))[0]

        # Save the binary image
        binary_image_path = os.path.join(output_folder, f'{filename}.jpg')
        cv2.imwrite(binary_image_path, binary_image)
        
    else:
        print(f"Image {image_path} not found or couldn't be loaded.")

# Path to the folder containing multiple images
input_folder = '2-final/'

# Output folder for processed images
output_folder = '3-final/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Process only image files
        image_path = os.path.join(input_folder, filename)
        process_image_binary(image_path, output_folder)

In [26]:
def process_image_morphology(image_path, output_folder):
    # Read the image in grayscale
    image = cv2.imread(image_path, 0)  # Read the image in grayscale
    # Define the kernel for morphological operations
    kernel = np.ones((3, 3), np.uint8)

    # Elliptical kernel
    kernel_ellipse = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (4, 4))

    # Cross-shaped kernel
    kernel_cross = cv2.getStructuringElement(cv2.MORPH_CROSS, (4, 4))

    # Apply opening to remove noise
    opening = cv2.morphologyEx(image, cv2.MORPH_OPEN, kernel)

    # Apply closing to fill holes
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel)

    # Get the filename without extension
    filename = os.path.splitext(os.path.basename(image_path))[0]

    # Save the processed image
    processed_image_path = os.path.join(output_folder, f'{filename}.png')
    cv2.imwrite(processed_image_path, closing)

# Path to the folder containing multiple images
input_folder = '3-final/'

# Output folder for processed images
output_folder = '4-final/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Process only image files
        image_path = os.path.join(input_folder, filename)
        process_image_morphology(image_path, output_folder)

In [28]:


def eliminar_pixeles_no_conectados(image_path, output_folder, umbral_conexion=50):

    imagen_binaria = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    # Etiquetar componentes conectados
    etiquetas, num_etiquetas = label(imagen_binaria)

    # Calcular el tamaño de cada componente
    tamaños = np.bincount(etiquetas.flatten())

    # Iterar sobre las etiquetas y eliminar píxeles no conectados
    for etiqueta in range(1, num_etiquetas + 1):
        if tamaños[etiqueta] <= umbral_conexion:
            # Eliminar píxeles no conectados
            imagen_binaria[etiquetas == etiqueta] = 0

    filename = os.path.splitext(os.path.basename(image_path))[0]

    # Save the processed image
    processed_image_path = os.path.join(output_folder, f'{filename}.png')
    cv2.imwrite(processed_image_path, imagen_binaria)

# Path to the folder containing multiple images
input_folder = '4-final/'

# Output folder for processed images
output_folder = '5-final/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Process each image in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.png'):  # Process only image files
        image_path = os.path.join(input_folder, filename)
        eliminar_pixeles_no_conectados(image_path, output_folder)

In [3]:
# Rutas de las carpetas
carpeta_inputs = "inputs/"
carpeta_final = "5-final/"
carpeta_salida = "6-final/"

if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)

# Obtener la lista de archivos en las carpetas
archivos_inputs = os.listdir(carpeta_inputs)
archivos_final = os.listdir(carpeta_final)

# Iterar sobre los archivos en la carpeta "inputs"
for archivo_input in archivos_inputs:
    # Comprobar si el archivo es una imagen
    if archivo_input.endswith(('.png', '.jpg', '.jpeg')):
        # Leer la imagen de input
        imagen_input = cv2.imread(os.path.join(carpeta_inputs, archivo_input))

        # Obtener el nombre del archivo sin extensión
        nombre_sin_extension = os.path.splitext(archivo_input)[0]

        # Buscar el archivo correspondiente en la carpeta "5-final"
        archivo_final = f"{nombre_sin_extension}.png"  # Puedes ajustar la extensión según tus archivos
        ruta_archivo_final = os.path.join(carpeta_final, archivo_final)

        # Comprobar si el archivo final existe
        if os.path.exists(ruta_archivo_final):
            # Leer la imagen binaria final
            mascara_final = cv2.imread(ruta_archivo_final, cv2.IMREAD_GRAYSCALE)

            # Aplicar la máscara a la imagen de input
            imagen_resultante = cv2.bitwise_and(imagen_input, imagen_input, mask=mascara_final)

            # Guardar la imagen resultante en la carpeta de salida
            ruta_salida = os.path.join(carpeta_salida, f"{nombre_sin_extension}.png")
            cv2.imwrite(ruta_salida, imagen_resultante)
        else:
            print(f"No se encontró la máscara correspondiente para {archivo_input}")

In [5]:
# Ruta de la carpeta que contiene las imágenes
carpeta_imagenes = "ground_truths"

# Obtener la lista de archivos en la carpeta
archivos_imagenes = os.listdir(carpeta_imagenes)

carpeta_salida = "ground_truths_w/"

if not os.path.exists(carpeta_salida):
    os.makedirs(carpeta_salida)

# Iterar sobre los archivos en la carpeta
for archivo_imagen in archivos_imagenes:
    # Comprobar si el archivo es una imagen
    if archivo_imagen.endswith(('.png', '.jpg', '.jpeg')):
        # Leer la imagen
        ruta_imagen = os.path.join(carpeta_imagenes, archivo_imagen)
        imagen = cv2.imread(ruta_imagen)

        # Convertir la imagen a escala de grises
        imagen_gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

        # Binarizar la imagen para obtener una máscara
        _, mascara = cv2.threshold(imagen_gris, 1, 255, cv2.THRESH_BINARY)

        # Crear una imagen blanca del mismo tamaño que la original
        imagen_blanca = np.ones_like(imagen) * 255

        # Copiar los píxeles de la imagen original a la imagen blanca donde la máscara es negra
        imagen_blanca[mascara == 0] = imagen[mascara == 0]

        # Guardar la imagen resultante
        ruta_salida = os.path.join(carpeta_salida, f"{archivo_imagen.split('.')[0]}.png")
        cv2.imwrite(ruta_salida, imagen_blanca)